<a href="https://colab.research.google.com/github/edumntg/OPF-python/blob/main/Pyomo_OPF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
#!pip install -q pyomo

In [76]:
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [77]:
from pyomo.environ import *
from pyomo.environ import (
    ConcreteModel, Set, Var, Reals, Objective, Constraint, minimize, SolverFactory, value
)
import numpy as np
from math import pi

# Data

In [78]:
Sbase = 10 # MW

buses = {
    1: [1, 0, 1.00, 0.0, 0.0, 0.0, 0.0, 0.0],
    2: [2, 1, 1.01, 0.0, 0.0, 0.0, 0.0, 0.0],
    3: [3, 2, 1.00, 0.0, 0.0, 0.0, 0.3, 0.1]
}

lines = {
    1: [1, 2, 0.0192, 0.0575, 0.0264, 1, 30/Sbase],
    2: [1, 3, 0.0452, 0.1852, 0.0204, 1, 30/Sbase],
    3: [2, 3, 0.0570, 0.1737, 0.0184, 1, 30/Sbase]
}

gens = {
    1: [1, 0/Sbase, 20/Sbase, -20/Sbase, 100/Sbase, 0.00375, 2, 0],
    2: [2, 0/Sbase, 20/Sbase, -20/Sbase, 100/Sbase, 0.0175, 1.75, 0]
}

# Create Ybus

In [79]:
nb = len(buses)
nl = len(lines)
ng = len(gens)

Ybus = np.zeros((nb, nb), dtype=np.complex128)
g = np.zeros((nb, nb))
b = np.zeros((nb,nb))
# Loop through lines
for lineid, linedata in lines.items():
  i = linedata[0]-1
  k = linedata[1]-1
  Z = linedata[2] + 1j*linedata[3]
  Bs = 1j*linedata[4]
  a = linedata[5]

  Ybus[i][i] += (1/(Z*a**2))
  Ybus[k][k] += (1/(Z*a**2))

  Ybus[i][i] += Bs
  Ybus[k][k] += Bs

  Ybus[i][k] -= 1/(a*Z)
  Ybus[k][i] -= 1/(a*Z)

  b[i][k] = Bs.imag
  b[k][i] = Bs.imag

G = Ybus.real
B = Ybus.imag

print(Ybus)
print(G)
print(B)

[[ 6.46838347-20.69594776j -5.22464618+15.64672684j
  -1.24373729 +5.09602092j]
 [-5.22464618+15.64672684j  6.9301765 -20.79930607j
  -1.70553032 +5.19737923j]
 [-1.24373729 +5.09602092j -1.70553032 +5.19737923j
   2.9492676 -10.25460015j]]
[[ 6.46838347 -5.22464618 -1.24373729]
 [-5.22464618  6.9301765  -1.70553032]
 [-1.24373729 -1.70553032  2.9492676 ]]
[[-20.69594776  15.64672684   5.09602092]
 [ 15.64672684 -20.79930607   5.19737923]
 [  5.09602092   5.19737923 -10.25460015]]


# Objective Function

In [80]:
def ObjectiveFunction(model):
  Cost = 0.0
  for genid, gendata in gens.items():
    bus = gendata[0]
    a = gendata[4]
    b = gendata[5]
    c = gendata[6]

    Cost += c*model.Pgen[bus]**2 + b*model.Pgen[bus] + a

  return Cost
  #return -sum(model.l[i] for i in model.bus)

# Constraints

In [81]:
def MinGen_P(model, bus):
  keys = [key for (key, v) in gens.items() if v[0] == bus]
  lb = 0
  if keys:
    lb = gens[keys[0]][1]
    
  return model.Pgen[bus] >= lb
  
def MaxGen_P(model, bus):
  keys = [key for (key, v) in gens.items() if v[0] == bus]
  ub = 0
  if keys:
    ub = gens[keys[0]][2]
    
  return model.Pgen[bus] <= ub

def MinGen_Q(model, bus):
  keys = [key for (key, v) in gens.items() if v[0] == bus]
  lb = 0
  if keys:
    lb = gens[keys[0]][3]
    
  return model.Qgen[bus] >= lb
  
def MaxGen_Q(model, bus):
  keys = [key for (key, v) in gens.items() if v[0] == bus]
  ub = 0
  if keys:
    ub = gens[keys[0]][4]
    
  return model.Qgen[bus] <= ub

def MaxFlowLineik(model, line):
  S = lines[line][6]
  i = lines[line][0]
  k = lines[line][1]
  return model.Pflow[i,k]**2+model.Qflow[i,k]**2 <= S

def MaxFlowLineki(model, line):
  S = lines[line][6]
  i = lines[line][1]
  k = lines[line][0]
  return model.Pflow[i,k]**2+model.Qflow[i,k]**2 <= S

def KirchoffBusesP(model, bus):
  Pik = 0
  Pgbus = model.Pgen[bus]
    
  for linea in model.line:
    i = lines[linea][0]
    if i == bus: # gen id is the same as bus id
      j = lines[linea][1]
      Pik += model.Pflow[i,j]
      
  for linea in model.line:
    i = lines[linea][1]
    if i == bus: # gen id is the same as bus id
      j = lines[linea][0]
      Pik += model.Pflow[i,j]
    
  return Pgbus == buses[bus][6] + Pik

def KirchoffBusesQ(model, bus):
  Qik = 0
  Qgbus = model.Qgen[bus]
  Qshunt = 0
  for linea in model.line:
    i = lines[linea][0]
    if i == bus: # gen id is the same as bus id
      j = lines[linea][1]
      Qik += model.Qflow[i,j]
      
  for linea in model.line:
    i = lines[linea][1]
    if i == bus: # gen id is the same as bus id
      j = lines[linea][0]
      Qik += model.Qflow[i,j]
      
    
  return Qgbus == buses[bus][7] + Qik + Qshunt
  
# Lines equations
def Pflow_square_like_original(model, linea):
    i = lines[linea][0]
    j = lines[linea][1]

    Vi2 = model.VR[i]**2 + model.VI[i]**2
    cos_term = model.VR[i]*model.VR[j] + model.VI[i]*model.VI[j]
    sin_term = model.VI[i]*model.VR[j] - model.VR[i]*model.VI[j]

    return model.Pflow[i, j] == (
        (-G[i-1][j-1] + g[i-1][j-1]) * Vi2
        + G[i-1][j-1] * cos_term
        + B[i-1][j-1] * sin_term
    )

def Qflow_square_like_original(model, linea):
    i = lines[linea][0]
    j = lines[linea][1]

    Vi2 = model.VR[i]**2 + model.VI[i]**2
    cos_term = model.VR[i]*model.VR[j] + model.VI[i]*model.VI[j]
    sin_term = model.VI[i]*model.VR[j] - model.VR[i]*model.VI[j]

    return model.Qflow[i, j] == (
        (B[i-1][j-1] - b[i-1][j-1]) * Vi2
        + (-B[i-1][j-1]) * cos_term
        + G[i-1][j-1] * sin_term
    )

def Pflow_square_like_original_rev(model, linea):
    i = lines[linea][1]
    j = lines[linea][0]
    Vi2 = model.VR[i]**2 + model.VI[i]**2
    cos_term = model.VR[i]*model.VR[j] + model.VI[i]*model.VI[j]
    sin_term = model.VI[i]*model.VR[j] - model.VR[i]*model.VI[j]
    return model.Pflow[i, j] == (
        (-G[i-1][j-1] + g[i-1][j-1]) * Vi2
        + G[i-1][j-1] * cos_term
        + B[i-1][j-1] * sin_term
    )

def Qflow_square_like_original_rev(model, linea):
    i = lines[linea][1]
    j = lines[linea][0]
    Vi2 = model.VR[i]**2 + model.VI[i]**2
    cos_term = model.VR[i]*model.VR[j] + model.VI[i]*model.VI[j]
    sin_term = model.VI[i]*model.VR[j] - model.VR[i]*model.VI[j]
    return model.Qflow[i, j] == (
        (B[i-1][j-1] - b[i-1][j-1]) * Vi2
        + (-B[i-1][j-1]) * cos_term
        + G[i-1][j-1] * sin_term
    )

def V_square_def(model, bus):
    return model.Vsq[bus] == model.VR[bus]**2 + model.VI[bus]**2



# Finally, solve

In [82]:
model = ConcreteModel()

model.bus = Set(initialize = buses.keys())
model.line = Set(initialize = lines.keys())
model.gen = Set(initialize = gens.keys())

# Create variables
model.Pgen = Var(model.bus, initialize = 0)
model.Qgen = Var(model.bus, initialize = 0)
model.VR = Var(model.bus, initialize = 1.0, bounds = (-1.1, 1.1))
model.VI = Var(model.bus, initialize = 1.0, bounds = (-1.1, 1.1))
model.Vsq = Var(model.bus, initialize = 1.0, bounds = (0.81, 1.21))

# Line flows
model.Pflow = Var(model.bus, model.bus)
model.Qflow = Var(model.bus, model.bus)

model.obj = Objective(rule = ObjectiveFunction, sense = minimize)

model.c0 = Constraint(expr = model.VI[1] == 0)

model.c1 = Constraint(model.bus, rule = KirchoffBusesP)
model.c2 = Constraint(model.bus, rule = KirchoffBusesQ)

model.c3 = Constraint(model.bus, rule = MaxGen_P)
model.c4 = Constraint(model.bus, rule = MinGen_P)
model.c5 = Constraint(model.bus, rule = MaxGen_Q)
model.c6 = Constraint(model.bus, rule = MinGen_Q)


model.c7 = Constraint(model.line, rule = MaxFlowLineik)
model.c8 = Constraint(model.line, rule = MaxFlowLineki)

model.c9 = Constraint(model.line, rule = Pflow_square_like_original)
model.c10 = Constraint(model.line, rule = Pflow_square_like_original_rev)
model.c11 = Constraint(model.line, rule = Qflow_square_like_original)
model.c12 = Constraint(model.line, rule = Qflow_square_like_original_rev)


# Solve

In [83]:
import gurobipy
from pyomo.environ import *
print(gurobipy.gurobi.version())
#solver = SolverFactory('ipopt', executable='/content/ipopt')
solver = SolverFactory('gurobi_direct')
#solver = SolverFactory('cyipopt')
results = solver.solve(model)
print(results.solver.termination_condition)

(13, 0, 1)
optimal


In [84]:
def PrintOPFACResults(model, buses, lineas, gens, shunts):

	nb = len(buses)
	nl = len(lineas)
	ng = len(gens)
	ns = len(shunts)

	print('BusID	VR	VI	Pg	Qg	l	Pl	Ql	Qshunt\n')
	l = {}
	for i in model.bus:
		Qshunt = 0
		Pg = abs(model.Pgen[i]())
		Qg = model.Qgen[i]()
		
		print("{0:.0f}	{1:.4f}	{2:.4f}	{3:.4f}	{4:.4f}	{5:.4f}	{6:.4f}	{7:.4f}	{8:.4f}".format(i,model.VR[i](),model.VI[i](),Pg,Qg, 1.0, buses[i][6], buses[i][7], 0.0))

	Pgtotal = sum(model.Pgen[i]() for i in model.bus)
	Qgtotal = sum(model.Qgen[i]() for i in model.bus)

	Ploadtotal = sum(buses[i][6] for i in model.bus)
	Qloadtotal = sum(buses[i][7] for i in model.bus)

	print("\n")
	print("TOTAL			{0:.4f}	{1:.4f}		{2:.4f}	{3:.4f}".format(Pgtotal, Qgtotal, Ploadtotal, Qloadtotal))
	print("\n\n")

	print("Busi	Busk	Pik	Pki	Qik	Qki")
	Pik = np.zeros((nb,nb))
	Pki = np.zeros((nb,nb))
	Qik = np.zeros((nb,nb))
	Qki = np.zeros((nb,nb))

	for l in model.line:
		i = lineas[l][0]
		j = lineas[l][1]
		print("{0:.0f}	{1:.0f}	{2:.4f}	{3:.4f}	{4:.4f}	{5:.4f}".format(i,j,model.Pflow[i,j](),model.Pflow[j,i](),model.Qflow[i,j](),model.Qflow[j,i]()))
		
	Ploss = 0
	Qloss = 0
	for l in model.line:
		i = lineas[l][0]
		j = lineas[l][1]
		Ploss += model.Pflow[i,j]() + model.Pflow[j,i]()
		Qloss += model.Qflow[i,j]() + model.Qflow[j,i]()


	Pl_supplied = sum(buses[i][6] for i in model.bus)
	Pl_total = sum(buses[i][6] for i in model.bus)
	perc_supplied = (Pl_supplied/Pl_total)*100
	
	print("\n")
	print("Total Ploss: {0:.4f}\nTotal Qloss: {1:.4f}".format(Ploss,Qloss))
	print("Total Load Supplied: {0:.4f}%".format(perc_supplied))

In [85]:
PrintOPFACResults(model, buses, lines, gens, [])

BusID	VR	VI	Pg	Qg	l	Pl	Ql	Qshunt

1	1.1000	0.0000	0.1428	-0.0201	1.0000	0.0000	0.0000	0.0000
2	1.1000	0.0002	0.1592	-0.0297	1.0000	0.0000	0.0000	0.0000
3	1.0879	-0.0231	0.0000	0.0000	1.0000	0.3000	0.1000	0.0000


TOTAL			0.3020	-0.0497		0.3000	0.1000



Busi	Busk	Pik	Pki	Qik	Qki
1	2	-0.0033	0.0033	-0.0312	-0.0326
1	3	0.1461	-0.1453	0.0112	-0.0565
2	3	0.1559	-0.1547	0.0030	-0.0435


Total Ploss: 0.0020
Total Qloss: -0.1497
Total Load Supplied: 100.0000%


In [86]:
import numpy as np
from pyomo.environ import (
    ConcreteModel, Set, Var, Reals, Objective, Constraint, minimize, SolverFactory, value
)

# -----------------------------
# Data (same structure as your notebook)
# -----------------------------
Sbase = 10  # MW

buses = {
    1: [1, 0, 1.00, 0.0, 0.0, 0.0, 0.0, 0.0],
    2: [2, 1, 1.01, 0.0, 0.0, 0.0, 0.0, 0.0],
    3: [3, 2, 1.00, 0.0, 0.0, 0.0, 0.3, 0.1],  # Pd=0.3, Qd=0.1 (p.u. on Sbase)
}

lines = {
    1: [1, 2, 0.0192, 0.0575, 0.0264, 1, 30 / Sbase],
    2: [1, 3, 0.0452, 0.1852, 0.0204, 1, 30 / Sbase],
    3: [2, 3, 0.0570, 0.1737, 0.0184, 1, 30 / Sbase],
}

gens = {
    1: [1, 0 / Sbase, 20 / Sbase, -20 / Sbase, 100 / Sbase, 0.00375, 2.0, 0.0],
    2: [2, 0 / Sbase, 20 / Sbase, -20 / Sbase, 100 / Sbase, 0.0175, 1.75, 0.0],
}
# gens[g] = [bus, Pmin, Pmax, Qmin, Qmax, c2, c1, c0]

# voltage magnitude bounds (like your notebook)
Vmin, Vmax = 0.9, 1.1
Vmin2, Vmax2 = Vmin**2, Vmax**2

# Build adjacency / neighbor sets from undirected lines
bus_ids = sorted(buses.keys())
line_ids = sorted(lines.keys())

neighbors = {i: set() for i in bus_ids}
dir_arcs = []  # directed arcs (i,k) for each undirected line => (i,k) and (k,i)
for ell in line_ids:
    i, k = lines[ell][0], lines[ell][1]
    neighbors[i].add(k)
    neighbors[k].add(i)
    dir_arcs.append((i, k))
    dir_arcs.append((k, i))

# Map arc -> its parent line id (undirected)
arc_to_line = {}
for ell in line_ids:
    i, k = lines[ell][0], lines[ell][1]
    arc_to_line[(i, k)] = ell
    arc_to_line[(k, i)] = ell

# Helper: series admittance from r,x
def series_admittance(r, x):
    z2 = r*r + x*x
    g = r / z2
    b = -x / z2  # series susceptance (note sign)
    return g, b

# -----------------------------
# Pyomo model (Rectangular ACOPF QCQP)
# -----------------------------
m = ConcreteModel()
m.BUS = Set(initialize=bus_ids)
m.LINE = Set(initialize=line_ids)
m.ARC = Set(initialize=dir_arcs, dimen=2)

# Voltage in rectangular coordinates: V = e + j f
m.e = Var(m.BUS, domain=Reals, initialize=lambda mm, i: buses[i][2])  # start at |V|
m.f = Var(m.BUS, domain=Reals, initialize=0.0)

# Generator injections (defined for all buses; non-gen buses fixed to 0 via constraints)
m.Pg = Var(m.BUS, domain=Reals, initialize=0.0)
m.Qg = Var(m.BUS, domain=Reals, initialize=0.0)

# Line flows (directed)
m.P = Var(m.ARC, domain=Reals, initialize=0.0)
m.Q = Var(m.ARC, domain=Reals, initialize=0.0)

# Objective: sum over generators: c2*Pg^2 + c1*Pg + c0
gen_buses = set(gens[g][0] for g in gens)

def obj_rule(mm):
    cost = 0.0
    for g, gd in gens.items():
        b = gd[0]
        c2, c1, c0 = gd[5], gd[6], gd[7]
        cost += c2 * mm.Pg[b] ** 2 + c1 * mm.Pg[b] + c0
    return cost

m.obj = Objective(rule=obj_rule, sense=minimize)

# Slack/reference bus: fix angle = 0 by setting f=0, and fix magnitude by e=Vref
ref_bus = 1
Vref = buses[ref_bus][2]
m.ref1 = Constraint(expr=m.f[ref_bus] == 0.0)
m.ref2 = Constraint(expr=m.e[ref_bus] == Vref)

# Voltage magnitude bounds: Vmin^2 <= e^2 + f^2 <= Vmax^2
def vmag_lb_rule(mm, i):
    return mm.e[i]**2 + mm.f[i]**2 >= Vmin2
def vmag_ub_rule(mm, i):
    return mm.e[i]**2 + mm.f[i]**2 <= Vmax2

m.vmag_lb = Constraint(m.BUS, rule=vmag_lb_rule)
m.vmag_ub = Constraint(m.BUS, rule=vmag_ub_rule)

# Generator bounds + force non-generator buses to Pg=Qg=0
def gen_bounds_rule(mm, i):
    if i in gen_buses:
        # find the gen record for this bus (assumes <=1 gen per bus like your data)
        gkey = next(k for k, gd in gens.items() if gd[0] == i)
        Pmin, Pmax, Qmin, Qmax = gens[gkey][1], gens[gkey][2], gens[gkey][3], gens[gkey][4]
        return (Pmin, mm.Pg[i], Pmax)
    else:
        return (0.0, mm.Pg[i], 0.0)

def gen_bounds_Q_rule(mm, i):
    if i in gen_buses:
        gkey = next(k for k, gd in gens.items() if gd[0] == i)
        Qmin, Qmax = gens[gkey][3], gens[gkey][4]
        return (Qmin, mm.Qg[i], Qmax)
    else:
        return (0.0, mm.Qg[i], 0.0)

m.Pg_bounds = Constraint(m.BUS, rule=gen_bounds_rule)
m.Qg_bounds = Constraint(m.BUS, rule=gen_bounds_Q_rule)

# Directed branch flow equations (pi-model with line charging b_shunt/2 at each end)
# Using (e,f) formulation; tap ratio a is in data but equals 1 here.
def branch_flow_P_rule(mm, i, k):
    ell = arc_to_line[(i, k)]
    _, _, r, x, b_shunt, a, _ = lines[ell]
    g, b = series_admittance(r, x)
    a = float(a)

    ei, fi = mm.e[i], mm.f[i]
    ek, fk = mm.e[k], mm.f[k]
    Vi2 = ei**2 + fi**2

    # P_ik = (g/a^2)*|Vi|^2 - (g/a)*(ei*ek+fi*fk) - (b/a)*(fi*ek - ei*fk)
    return mm.P[i, k] == (g/(a*a))*Vi2 - (g/a)*(ei*ek + fi*fk) - (b/a)*(fi*ek - ei*fk)

def branch_flow_Q_rule(mm, i, k):
    ell = arc_to_line[(i, k)]
    _, _, r, x, b_shunt, a, _ = lines[ell]
    g, b = series_admittance(r, x)
    a = float(a)

    ei, fi = mm.e[i], mm.f[i]
    ek, fk = mm.e[k], mm.f[k]
    Vi2 = ei**2 + fi**2

    # Q_ik = -(b/a^2)*|Vi|^2 - (b/a)*(ei*ek+fi*fk) + (g/a)*(fi*ek - ei*fk) + (b_shunt/2a^2)*|Vi|^2
    # (line charging contributes +j*b_shunt/2 at each end)
    return mm.Q[i, k] == (-(b/(a*a))*Vi2
                          - (b/a)*(ei*ek + fi*fk)
                          + (g/a)*(fi*ek - ei*fk)
                          + (b_shunt/(2*a*a))*Vi2)

m.branchP = Constraint(m.ARC, rule=branch_flow_P_rule)
m.branchQ = Constraint(m.ARC, rule=branch_flow_Q_rule)

# Line MVA (actually |S|^2) limits for each direction:
# P_ik^2 + Q_ik^2 <= Smax^2   (your notebook used <= S; that was dimensionally inconsistent)
def thermal_limit_rule(mm, ell):
    i, k, *_ = lines[ell]
    Smax = lines[ell][6]
    return (mm.P[i, k]**2 + mm.Q[i, k]**2 <= Smax**2)

def thermal_limit_rev_rule(mm, ell):
    i, k, *_ = lines[ell]
    Smax = lines[ell][6]
    return (mm.P[k, i]**2 + mm.Q[k, i]**2 <= Smax**2)

m.therm_fwd = Constraint(m.LINE, rule=thermal_limit_rule)
m.therm_rev = Constraint(m.LINE, rule=thermal_limit_rev_rule)

# Power balance at each bus: Pg - Pd = sum_k P_ik over incident neighbors
# Qg - Qd = sum_k Q_ik
def p_balance_rule(mm, i):
    Pd = buses[i][6]
    return mm.Pg[i] - Pd == sum(mm.P[i, k] for k in neighbors[i])

def q_balance_rule(mm, i):
    Qd = buses[i][7]
    return mm.Qg[i] - Qd == sum(mm.Q[i, k] for k in neighbors[i])

m.pbal = Constraint(m.BUS, rule=p_balance_rule)
m.qbal = Constraint(m.BUS, rule=q_balance_rule)

# -----------------------------
# Solve with Gurobi (nonconvex QCQP)
# -----------------------------
solver = SolverFactory("gurobi")
solver.options["NonConvex"] = 2  # REQUIRED for nonconvex quadratic constraints
# Optional:
# solver.options["MIPGap"] = 1e-6
# solver.options["TimeLimit"] = 60

res = solver.solve(m, tee=True)

print("\nTermination:", res.solver.termination_condition)
print("Objective:", value(m.obj))
for i in m.BUS:
    print(f"Bus {i}: e={value(m.e[i]): .6f}, f={value(m.f[i]): .6f}, "
          f"Pg={value(m.Pg[i]): .6f}, Qg={value(m.Qg[i]): .6f}, "
          f"|V|={np.sqrt(value(m.e[i]**2 + m.f[i]**2)): .6f}")

for ell in m.LINE:
    i, k = lines[ell][0], lines[ell][1]
    print(f"Line {ell} {i}->{k}: P={value(m.P[i,k]): .6f}, Q={value(m.Q[i,k]): .6f}")
    print(f"Line {ell} {k}->{i}: P={value(m.P[k,i]): .6f}, Q={value(m.Q[k,i]): .6f}")


Read LP format model from file C:\Users\zzhao3\AppData\Local\Temp\1\tmpgum2ysf9.pyomo.lp
Reading time = 0.00 seconds
x1: 18 rows, 24 columns, 30 nonzeros
Set parameter NonConvex to value 2
Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
NonConvex  2

Optimize a model with 18 rows, 24 columns and 30 nonzeros (Min)
Model fingerprint: 0x505f5f22
Model has 2 linear objective coefficients
Model has 2 quadratic objective terms
Model has 24 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 2e+01]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [2e+00, 2e+00]
  QObjective range [7e-03, 4e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+01]
  QRHS range       [8e-01, 9e+00]

Presolve removed 12 r